In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, models, losses

https://www.youtube.com/watch?v=0me3guauqOU

In [2]:
# init x and y datasets
X = []
y = []

# images of apples (encoded as 0)
all_images = [f for f in os.listdir('apple_images') if isfile(join('apple_images', f))]

for file in all_images:
    img_arr=cv2.imread(f'apple_images/{file}')
    img_arr=cv2.resize(img_arr,(256,256))
    X.append(img_arr)
    y.append(0)

In [3]:
# images of bananas (encoded as 1)
all_images = [f for f in os.listdir('banana_images') if isfile(join('banana_images', f))]

for file in all_images:
    img_arr=cv2.imread(f'banana_images/{file}')
    img_arr=cv2.resize(img_arr,(256,256))
    X.append(img_arr)
    y.append(1)

In [4]:
# convert datasets to numpy arrays, scale RGB values down by 255
X_scaled = np.array(X)/255
y = np.array(y)

In [5]:
# training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=1)

In [6]:
# build model architecture
# model based on model seen at https://medium.com/analytics-vidhya/lenet-with-tensorflow-a35da0d503df
model = models.Sequential()

# First convolution/pooling
model.add(layers.Conv2D(6, 5, activation='relu', input_shape=X_train.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.LeakyReLU(alpha=0.1))

# Second convolution/pooling
model.add(layers.Conv2D(16, 5, activation='relu'))
model.add(layers.AveragePooling2D(2))
model.add(layers.LeakyReLU(alpha=0.1))

# Third convolution
model.add(layers.Conv2D(120, 5, activation='relu'))
model.add(layers.Flatten())

# Binary classifier
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 252, 252, 6)       456       
                                                                 
 average_pooling2d (AverageP  (None, 126, 126, 6)      0         
 ooling2D)                                                       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 126, 126, 6)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 16)      2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 61, 61, 16)       0         
 ePooling2D)                                                     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 61, 61, 16)        0

In [7]:
# compile and fit the model
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
123/123 [==============================] - 5s 16ms/step - loss: 0.5684 - accuracy: 0.8537
Epoch 2/10
123/123 [==============================] - 2s 15ms/step - loss: 0.5933 - accuracy: 0.9268
Epoch 3/10
123/123 [==============================] - 2s 15ms/step - loss: 0.0579 - accuracy: 0.9756 
Epoch 4/10
123/123 [==============================] - 2s 16ms/step - loss: 0.0828 - accuracy: 0.9756
Epoch 5/10
123/123 [==============================] - 2s 16ms/step - loss: 0.2497 - accuracy: 0.9756
Epoch 6/10
123/123 [==============================] - 2s 16ms/step - loss: 0.1807 - accuracy: 0.9675
Epoch 7/10
123/123 [==============================] - 2s 16ms/step - loss: 0.1315 - accuracy: 0.9268
Epoch 8/10
123/123 [==============================] - 2s 16ms/step - loss: 0.1890 - accuracy: 0.9675
Epoch 9/10
123/123 [==============================] - 2s 15ms/step - loss: 0.2247 - accuracy: 0.9350
Epoch 10/10
123/123 [==============================] - 2s 16ms/step - loss: 0.1814 - accur

In [8]:
# perform predictions
predictions = model.predict(X_test)

In [9]:
# reverse endcoding from probablity to binary value
predictions = pd.DataFrame(predictions)
predictions['P'] = predictions.idxmax(axis = 1)
predictions['A'] = y_test
predictions.head()

,0,1,P,A
0,1.000000e+00,3.218031e-15,0,0
1,1.000000e+00,2.567998e-15,0,0
2,2.978402e-05,9.999702e-01,1,1
3,1.000000e+00,2.925404e-16,0,0
4,4.740054e-09,1.000000e+00,1,1


In [10]:
# view model performance
print (classification_report(y_test, predictions['P']))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90        22
           1       1.00      0.75      0.86        20

    accuracy                           0.88        42
   macro avg       0.91      0.88      0.88        42
weighted avg       0.90      0.88      0.88        42



In [11]:
apple=cv2.imread(f'test_images/lots_of_apples.jpg')
apple=cv2.resize(apple,(256,256))
apple = np.array(apple)
apple = tf.expand_dims(apple, axis = 0)
apple.shape

TensorShape([1, 256, 256, 3])

In [12]:
apple_prediction = print(model.predict(apple))

[[0. 1.]]


In [13]:
pd.DataFrame(apple_prediction)

""
